### Import packages and get access to the training text file

In [ ]:
### For colab usage
from google.colab import drive
drive.mount('/content/drive')

%cd drive/My Drive/Colab Notebooks/HW4

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks/HW4


In [ ]:
!pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 9.6 MB/s eta 0:00:00


In [ ]:
import unidecode
import string
import random
import time
import math

import torch
import torch.nn as nn
from torch.autograd import Variable
import argparse
import os

from tqdm import tqdm

In [ ]:
### helpers.py

def read_file(filename):
    file = unidecode.unidecode(open(filename).read())
    return file, len(file)

filename = 'medline.0.txt'
file, file_len = read_file(filename)

### Functions to create the model

In [ ]:
### train.py
### corrected one mistake in train of using cuda
### corrected one mistake in the last line of train: previous code is loss.data[0]
### which causes error, so changed to loss

def random_training_set(chunk_len, batch_size):
    inp = torch.LongTensor(batch_size, chunk_len)
    target = torch.LongTensor(batch_size, chunk_len)
    for bi in range(batch_size):
        start_index = random.randint(0, file_len - chunk_len)
        end_index = start_index + chunk_len + 1
        chunk = file[start_index:end_index]
        inp[bi] = char_tensor(chunk[:-1])
        target[bi] = char_tensor(chunk[1:])
    inp = Variable(inp)
    target = Variable(target)
    if cuda:
        inp = inp.cuda()
        target = target.cuda()
    return inp, target

def train(inp, target):
    hidden = decoder.init_hidden(batch_size)

    if cuda:
        #Can't convert hidden to cuda because hidden is a tuple of tensor, not tensor.
        #Need to convert it to list, then convert each of the elements to cuda,
        #then convert back to a tuple.
        hidden = list(hidden)
        hidden[0] = hidden[0].cuda()
        hidden[1] = hidden[1].cuda()
        hidden = tuple(hidden)
        #hidden = hidden.cuda()

    decoder.zero_grad()
    loss = 0

    for c in range(chunk_len):
        output, hidden = decoder(inp[:,c], hidden)
        loss += criterion(output.view(batch_size, -1), target[:,c])

    loss.backward()
    decoder_optimizer.step()

    return loss / chunk_len

def save():
    save_filename = os.path.splitext(os.path.basename(filename))[0] + '.pt'
    torch.save(decoder, save_filename)
    print('Saved as %s' % save_filename)

In [ ]:
### model.py

class CharRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, model="gru", n_layers=1):
        super(CharRNN, self).__init__()
        self.model = model.lower()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers

        self.encoder = nn.Embedding(input_size, hidden_size)
        if self.model == "gru":
            self.rnn = nn.GRU(hidden_size, hidden_size, n_layers)
        elif self.model == "lstm":
            self.rnn = nn.LSTM(hidden_size, hidden_size, n_layers)
            #self.rnn = nn.LSTM(hidden_size, hidden_size, n_layers, bias=False) #try to set bias as False
        self.decoder = nn.Linear(hidden_size, output_size)

    def forward(self, input, hidden):
        batch_size = input.size(0)
        encoded = self.encoder(input)
        output, hidden = self.rnn(encoded.view(1, batch_size, -1), hidden)
        output = self.decoder(output.view(batch_size, -1))
        return output, hidden

    def forward2(self, input, hidden):
        encoded = self.encoder(input.view(1, -1))
        output, hidden = self.rnn(encoded.view(1, 1, -1), hidden)
        output = self.decoder(output.view(1, -1))
        return output, hidden

    def init_hidden(self, batch_size):
        if self.model == "lstm":
            return (Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size)),
                    Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size)))
        return Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size))

In [ ]:
### generate.py
### corrected one mistake of using cuda, the same as in train

def generate(decoder, prime_str='A', predict_len=100, temperature=0.8, cuda=False):
    hidden = decoder.init_hidden(1)
    prime_input = Variable(char_tensor(prime_str).unsqueeze(0))

    if cuda:
      # using LSTM: tuple tensors
        hidden = list(hidden)
        hidden[0] = hidden[0].cuda()
        hidden[1] = hidden[1].cuda()
        hidden = tuple(hidden)
        #hidden = hidden.cuda()
        prime_input = prime_input.cuda()
    predicted = prime_str

    # Use priming string to "build up" hidden state
    for p in range(len(prime_str) - 1):
        _, hidden = decoder.forward2(prime_input[:,p], hidden)

    inp = prime_input[:,-1]

    for p in range(predict_len):
        output, hidden = decoder.forward2(inp, hidden)

        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]

        # Add predicted character to string and use as next input
        predicted_char = all_characters[top_i]
        predicted += predicted_char
        inp = Variable(char_tensor(predicted_char).unsqueeze(0))
        if cuda:
            inp = inp.cuda()

    return predicted

In [ ]:
### healpers.py

def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        try:
            tensor[c] = all_characters.index(string[c])
        except:
            continue
    return tensor

### Training and Generate

In [ ]:
all_characters = string.printable
n_characters = len(all_characters)
n_layers = 4
n_epochs = 2000

hidden_size = 100
batch_size = 100
learning_rate = 0.01
model = "lstm"

chunk_len = 500
print_every = 100
cuda = True

In [ ]:
decoder = CharRNN(
    n_characters,
    hidden_size,
    n_characters,
    model=model,
    n_layers=n_layers,
)

if cuda:
    decoder.cuda()

decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

start = time.time()
all_losses = []
loss_avg = 0

In [ ]:
for epoch in tqdm(range(1, n_epochs + 1)):
    loss = train(*random_training_set(chunk_len, batch_size))
    loss_avg += loss

    if epoch % print_every == 0:
        print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 100, loss))
        print(generate(decoder, 'Wh', 100, cuda=cuda), '\n')

print("Saving...")
save()

  5%|▍         | 99/2000 [03:01<1:00:40,  1.92s/it]

 3s (100 5%) 3.2963]


  5%|▌         | 100/2000 [03:02<1:00:36,  1.91s/it]

Who r ut n tP89tde
0teg2(0An9a2 Aso Mc/nitc s cec ter 
:ay,.Ho0Rta S9(3nt  r rt o1AAet Blass wAUaT2ge( 



 10%|█         | 200/2000 [06:00<50:41,  1.69s/it]

...

 95%|█████████▌| 1900/2000 [56:54<03:10,  1.91s/it]

 55s (1900 95%) 1.0381]
Wh M-Mille Cells
JID - 101225837
RN  - 0 (Antineoplastic Agents)
RN  - 0 (RNA, Medicine, Immunokinetic 



100%|██████████| 2000/2000 [1:20:50<00:00,  2.43s/it]

 51s (2000 100%) 0.9141]
Whased There platin & cell lesion more and specialite and
      this volume at developing tumor cit bl 

Saving...


Saved as medline.0.pt


In [ ]:
# using 2 layers
generate(decoder=torch.load("medline.0.pt"), prime_str="PMID", predict_len=2000, temperature=0.8, cuda=True)

<ipython-input-37-87efb78ee750>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  generate(decoder=torch.load("medline.0.pt"), prime_str="PMID", predict_len=2000, temperature

"PMID- Non-Sp)m Sci.490-512. doi orged head tissues. Activator cells.\nPG  - 32-68\nLID - 10.1016/j.dpp.202200978 [doi]\nAB  - BACKGROUND: Patussay. The locology, Facing and exama and significant internal better in\n      concence analysis of the gene powen of 62.7%, p = 0.591 % (Darraielzy\nAU  - Kim Hormonn TS\nAD  - Department of Cancer and France. Amiliation of the European Social Several Amages of the\n      University Institute of Biology de Human Pharmacysig. We University a\n      regimatory as selective status and differentially was a positive\n      improved eachimetins have developments were embraint \n      constude are results were : induction and studies rearran pancreatic cancer carcinoma\n      staged seculation on breast percutoned by our patient death) = 0.79 months\n      those reactivity.\nFAU - Poto, Rodenanica\nAU  - Chun MY\nFAU - Tang, Perpavitie\nAU  - Arettol G\nLA  - eng\nPT  - Journal Article\nPT  - Research Support, Non-U.S. Gov't\nPT  - Review\nPL  - Unite

In [ ]:
# using 4 layers
generate(decoder=torch.load("medline.0.pt"), prime_str="PMID", predict_len=2000, temperature=0.8, cuda=True)

<ipython-input-15-059a012d4bcf>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  generate(decoder=torch.load("medline.0.pt"), prime_str="PMID", predict_len=2000, temperature

'PMID- 23009514\nOWN - NLM\nSTAT- MEDLINE\nDCOM- 20130413\nLR  - 20170220\nIS  - 1742-5100 (Electronic)\nIS  - 1679-1947 (Linking)\nVI  - 23\nIP  - 1\nDP  - 2013 Jan 23\nTI  - The on lice and certicipate that breast cancer temosis to classified\n      to statisticity stent risk and analyses, involvement of metastases in response of \n      a valuable cancer in the significantly and contributeal study secitant and reduction, the rall,\n      cancer conventional prostate status.\nPG  - 839-74\nLID - 10.1002/cncr.2330 [doi]\nAB  - Chronic based the prostate cancer activation and anticancer factors to Jatude to\n      temiate to the arogen these immunohistochemical sensitives were genes for the nanostatistically\n      risk of 27 patients with a sequencing therapy of the completed proteins, differention-associated\n      protein biomarkers. Eureton perithents with has estelners.\nFAU - Forecands Patrunn-Ah. Crind\nAU  - Kaman Jan WJ\nLA  - eng\nPT  - Journal Article\nDEP - 20120920\nPL  - 

In [ ]:
fake_paragraph_1 = generate(decoder=torch.load("medline.0.pt"), prime_str="PMID", predict_len=2000, temperature=0.8, cuda=True)
fake_paragraph_2 = generate(decoder=torch.load("medline.0.pt"), prime_str="PMID", predict_len=2000, temperature=0.8, cuda=True)
fake_paragraph_3 = generate(decoder=torch.load("medline.0.pt"), prime_str="PMID", predict_len=2000, temperature=0.8, cuda=True)

<ipython-input-16-b5ff4af01f7f>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  fake_paragraph_1 = generate(decoder=torch.load("medline.0.pt"), prime_str="PMID", predict_le

In [ ]:
real_paragraph = file[:2000]
import random
paragraphs = [fake_paragraph_1, fake_paragraph_2, fake_paragraph_3, real_paragraph]
random.shuffle(paragraphs)

# Step 5: Write the shuffled paragraphs to a file
with open('Li-TingKu4.MedLine.Sample.txt', 'w') as f:
    for i, paragraph in enumerate(paragraphs):
        f.write(f"Paragraph {i + 1}:\n")
        f.write(paragraph + "\n\n")

## Summary

Instead of using the command line, I made them into the python script together so I could use them more clearly. I first found that there are two forward method in the CharRNN model, so I made the `forward` into the training process with batches of sequences, whereas `forward2` be used into the generating process for sequential character generation. Then I select the LSTM model which is more complex then GRU because the pubmed abstract seems to be a complex text. In the training process, I want to try wth different layers to increase the model capacity and to learn complex patterns. I used 2 layers with 2000 epoch and 4 layers with 2000 epoch to see if there will be some difference, but I feel like the two result gave me similar structure of the pubmed abstract output, and when reading the context in the two results, I think using 4 layers generate more reasonable context compared to 2 layers.
